# Final Project

## Step 1: get and prepare the data

In [1]:
# Set up the libraries
import h2o
import pandas as pd
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)56-b12)
  Starting server from C:\Users\a.nogue.sanchez\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\ANOGUE~1.SAN\AppData\Local\Temp\tmp1nygi_0t
  JVM stdout: C:\Users\ANOGUE~1.SAN\AppData\Local\Temp\tmp1nygi_0t\h2o_a_nogue_sanchez_started_from_python.out
  JVM stderr: C:\Users\ANOGUE~1.SAN\AppData\Local\Temp\tmp1nygi_0t\h2o_a_nogue_sanchez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,4 months and 13 days !!!
H2O cluster name:,H2O_from_python_a_nogue_sanchez_gh0x7x
H2O cluster total nodes:,1
H2O cluster free memory:,3.533 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [2]:
# Load the data
data = h2o.import_file("http://coursera.h2o.ai/house_data.3487.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
# Convert date into year and month
data = data.as_data_frame() # didn't manage to work with H2O frames, sorry for that
data['year'] = [data['date'][i][:4] for i in range(len(data))]
data['month'] = [data['date'][i][4:6] for i in range(len(data))]
data = h2o.H2OFrame(data)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# Split the data
train, test = data.split_frame([0.9], seed = 123)
print("Train length: %d / Test length: %d" %(train.nrows, test.nrows))

# Split again to get a validation set
train, valid = train.split_frame([0.85], seed = 123)
print("Train length: %d / Test length: %d / Valid length: %d" %(train.nrows, test.nrows, valid.nrows))

Train length: 19462 / Test length: 2151
Train length: 16584 / Test length: 2151 / Valid length: 2878


In [5]:
# Specify the x and y variables
y = 'price'
ignoreFields = ['id', 'date', 'price', 'year', 'month']
x = [i for i in data.names if i not in ignoreFields]

## Step 2: training fours models

## Generalized Linear Model

In [58]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [90]:
my_GBM = H2OGradientBoostingEstimator(
    model_id = 'my_GBM',
    ntrees = 1000,
    max_depth = 4,
    seed = 123
    )
my_GBM.train(x, y, train, validation_frame = valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [ ]:
from h2o.grid.grid_search import H2OGridSearch
hyper_parameters = { 'ntrees': [100, 400, 700, 1000, 2000],
                     'max_depth' : [4, 6, 7, 8, 10],
                     'learn_rate' : [0.01, 0.03, 0.05, 0.1, 0.15],
                     'learn_rate_annealing' : [0.99, 1],
                     'distribution' : ['gaussian', 'poisson']}

search_criteria = { 'strategy': "RandomDiscrete", 'seed': 123}

random_plus_manual = H2OGridSearch(H2OGradientBoostingEstimator(),
      hyper_parameters, 
      search_criteria=search_criteria)
    
random_plus_manual.train(x=x,y=y, training_frame=train, validation_frame = valid)

gbm Grid Build progress: |████████████████████████████████████████████████

In [87]:
gbm_gridperf1 = random_plus_manual.get_grid(sort_by='rmse', decreasing=True)
gbm_gridperf1

     max_depth ntrees  \
0           16   1000   
1           20   1000   
2           12   1000   
3            4    100   
4           16    100   
5           20    100   
6           12    100   
7            6    100   
8            8   1000   
9            8    100   
10           6   1000   
11           4   1000   

                                                        model_ids  \
0   Grid_GBM_py_7_sid_88da_model_python_1565462043260_14_model_11   
1    Grid_GBM_py_7_sid_88da_model_python_1565462043260_14_model_4   
2    Grid_GBM_py_7_sid_88da_model_python_1565462043260_14_model_2   
3    Grid_GBM_py_7_sid_88da_model_python_1565462043260_14_model_3   
4    Grid_GBM_py_7_sid_88da_model_python_1565462043260_14_model_7   
5   Grid_GBM_py_7_sid_88da_model_python_1565462043260_14_model_10   
6   Grid_GBM_py_7_sid_88da_model_python_1565462043260_14_model_12   
7    Grid_GBM_py_7_sid_88da_model_python_1565462043260_14_model_8   
8    Grid_GBM_py_7_sid_88da_model_python_156546204326

In [91]:
my_GBM.model_performance(valid)


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 15281698189.019205
RMSE: 123619.16594533068
MAE: 67118.52288865628
RMSLE: 0.1692573323102823
Mean Residual Deviance: 15281698189.019205
